In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2
import os
from sklearn.model_selection import KFold
num_folds=6

In [2]:
def image_to_feature_vector(image, size=(32, 32)):
	# resize the image to a fixed size, then flatten the image into
	# a list of raw pixel intensities
	return cv2.resize(image, size).flatten()

In [3]:
def extract_color_histogram(image, bins=(8, 8, 8)):
	# extract a 3D color histogram from the HSV color space using
	# the supplied number of `bins` per channel
	hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
	hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
		[0, 180, 0, 256, 0, 256])
	# handle normalizing the histogram if we are using OpenCV 2.4.X
	if imutils.is_cv2():
		hist = cv2.normalize(hist)
	# otherwise, perform "in place" normalization in OpenCV 3 (I
	# personally hate the way this is done
	else:
		cv2.normalize(hist, hist)
	# return the flattened histogram as the feature vector
	return hist.flatten()

In [4]:

datapath="C:/Users/20112/Desktop/keras-video-classification/keras-video-classification/data"
neighbors=1
jobs=-1


In [5]:
# grab the list of images that we'll be describing
print("[INFO] describing images...")
imagePaths = list(paths.list_images(datapath))
# initialize the raw pixel intensities matrix, the features matrix,
# and labels list
rawImages = []
features = []
labels = []

[INFO] describing images...


In [6]:
# loop over the input images
for (i, imagePath) in enumerate(imagePaths):
	# load the image and extract the class label (assuming that our
	# path as the format: /path/to/dataset/{class}.{image_num}.jpg
	image = cv2.imread(imagePath)
	label = imagePath.split(os.path.sep)[-2].split(".")[0]
	# extract raw pixel intensity "features", followed by a color
	# histogram to characterize the color distribution of the pixels
	# in the image
	pixels = image_to_feature_vector(image)
	hist = extract_color_histogram(image)
	# update the raw images, features, and labels matricies,
	# respectively
	rawImages.append(pixels)
	features.append(hist)
	labels.append(label)
	# show an update every 1,000 images
	if i > 0 and i % 100 == 0:
		print("[INFO] processed {}/{}".format(i, len(imagePaths)))

[INFO] processed 100/874
[INFO] processed 200/874
[INFO] processed 300/874
[INFO] processed 400/874
[INFO] processed 500/874
[INFO] processed 600/874
[INFO] processed 700/874
[INFO] processed 800/874


In [7]:
# show some information on the memory consumed by the raw images
# matrix and features matrix
rawImages = np.array(rawImages)
features = np.array(features)
labels = np.array(labels)
print("[INFO] pixels matrix: {:.2f}MB".format(
	rawImages.nbytes / (1024 * 1000.0)))
print("[INFO] features matrix: {:.2f}MB".format(
	features.nbytes / (1024 * 1000.0)))

[INFO] pixels matrix: 2.62MB
[INFO] features matrix: 1.75MB


In [8]:
# partition the data into training and testing splits, using 75%
# of the data for training and the remaining 25% for testing
(trainRI, testRI, trainRL, testRL) = train_test_split(
	rawImages, labels, test_size=0.25, random_state=42)
(trainFeat, testFeat, trainLabels, testLabels) = train_test_split(
	features, labels, test_size=0.25, random_state=42)
acc_per_fold = []
loss_per_fold = []
kfold = KFold(n_splits=num_folds, shuffle=True)

In [9]:
# train and evaluate a k-NN classifer on the raw pixel intensities
testArray=[]
fold_no = 1
for train, test in kfold.split(rawImages,labels):
	print("[INFO] evaluating raw pixel accuracy...")

	model = KNeighborsClassifier(n_neighbors=neighbors,
		n_jobs=jobs)
	model.fit(rawImages[train], labels[train])
	acc = model.score(rawImages[test], labels[test])
	print("[INFO] Raw pixel accuracy: {:.2f}%".format(acc * 100))
	

   


[INFO] evaluating raw pixel accuracy...
[INFO] Raw pixel accuracy: 65.41%
[INFO] evaluating raw pixel accuracy...
[INFO] Raw pixel accuracy: 61.17%
[INFO] evaluating raw pixel accuracy...
[INFO] Raw pixel accuracy: 62.89%


In [10]:

predictions = model.predict(testRI)
LABELS = set([ "qutub_minar","iron_pillar","alai_darwaza"])

print(classification_report(testLabels, predictions,
	target_names=LABELS))

              precision    recall  f1-score   support

 qutub_minar       0.88      0.82      0.85        34
 iron_pillar       0.95      0.87      0.91        95
alai_darwaza       0.85      0.94      0.89        90

    accuracy                           0.89       219
   macro avg       0.89      0.88      0.89       219
weighted avg       0.90      0.89      0.90       219

